In [ ]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [ ]:
import wandb

In [ ]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/yolov3_d53_mstrain-608_273e_coco.py')

root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/yolov3_d53_mstrain-608_273e_coco'

#cfg.model.roi_head.bbox_head.num_classes = 10
cfg.model.bbox_head.num_classes = 10


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)


## Wandb config
cfg.log_config = dict(
    interval=1000,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='WandbLoggerHook',
             init_kwargs=dict(
                 project='object-detection-pStage',
                 entity='dev-yewon',
                 name='Yolo-v3-01'
             )
            )        
    ])
# cfg.runner = dict(type='EpochBasedRunner', max_epochs=12)   # epoch 수 변경


In [ ]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

In [ ]:
# dataset 확인
datasets[0]

In [ ]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

In [ ]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)